# Structured output

It can be surprisingly tricky to retrieve the exact output format you want.  Here we'll try:
- simple prompt with unstructured text
- structured prompt that returns JSON and is parsed into Python

In [ ]:
import os
import json
from IPython.display import display, Markdown

from openai import OpenAI 
NRP_TOK = os.environ.get('NRP_TOK')

## Setting things up

In [ ]:
client = OpenAI(
    api_key = NRP_TOK,
    base_url = "https://ellm.nrp-nautilus.io/v1"
)

In [ ]:
# MODEL = "gpt-oss"
MODEL = "gemma3"

In [ ]:
BUG_REPORT = """
When I try to reset my password, I never receive the reset email.
I checked my spam folder and there is nothing there. This happens
for multiple users on our staging environment but works in production.
"""

## Basic and unstructured prompt

In [ ]:
user_prompt = f"""
Summarize the following bug report and suggest a severity level.

Bug report:
{BUG_REPORT}
"""

In [ ]:
user_prompt

In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful bug triage assistant.",
        },
        {
            "role": "user",
            "content": user_prompt,
        },
    ],
)

In [ ]:
response.choices[0].message.content

In [ ]:
display(Markdown(response.choices[0].message.content))

## More targeted and structured (JSON-only) prompt

In [ ]:
user_prompt = f"""
Read the bug report below and convert it into a JSON object with these fields:

- summary: one-sentence summary of the bug (string)
- severity: one of "low", "medium", "high" (string)
- area: short component/area name like "authentication", "email", or "billing" (string)

Requirements:
- Return VALID JSON ONLY.
- No markdown.
- No code fences.
- No extra commentary before or after the JSON.

Bug report:
{BUG_REPORT}
"""

In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": (
                "You are a bug triage assistant. "
                "You ALWAYS respond with a single JSON object and nothing else."
            ),
        },
        {
            "role": "user",
            "content": user_prompt,
        },
    ],
    # Turn on JSON mode so the model is constrained to output JSON
    # See OpenAI's Structured Outputs / JSON mode docs.
    response_format={"type": "json_object"}
)

print(response.choices[0].message.content)

## Programmatic use of output is more straight-forward now

In [ ]:
structured = response.choices[0].message.content

Raw model output should be JSON only:

In [ ]:
structured

We can try loading it with the `json` library:

In [ ]:
data = json.loads(structured)

It didn't error out on the `json.loads()` so seems good.

In [ ]:
print(json.dumps(data, indent=2))

And we can retrieve particular elements out of the response:

In [ ]:
data.get("summary")

In [ ]:
data.get("severity")

In [ ]:
data.get("area")

In [ ]:
print(f'''
Accessing individual fields:
    summary : {data.get("summary")}
    severity: {data.get("severity")}
    area    : {data.get("area")}
''')

# Larger example for Python debugging

In [ ]:
# note that you need {{ and }} if you want actual curly braces in the template

python_engineer_template = '''
You are a senior Python engineer.

TASK:
Review the following Python function and suggest improvements for readability and performance.

CONTEXT:
Here is the function:

{code}

CONSTRAINTS:
- Do not change the function’s external behavior.
- Keep the public interface (function name and parameters) the same.
- Use only the Python standard library.
- If the code is already good, say "No major issues found." and explain why.

EXAMPLES:
Example 1
Input code:
def add(a, b):
    return a + b

Desired output:
Issues:
- No major issues found.

Improved code:
def add(a: float, b: float) -> float:
    """Return the sum of a and b."""
    return a + b

Example 2
Input code:
def get_items(xs):
    res = []
    for i in range(0, len(xs)):
        res.append(xs[i])
    return res

Desired output:
Issues:
- Uses manual indexing instead of direct iteration.

Improved code:
def get_items(xs):
    return list(xs)

DESIRED OUTPUT FORMAT:
Respond in valid JSON with the following structure:

{{
  "issues": [
    "string description of issue 1",
    "string description of issue 2"
  ],
  "improved_code": "the full revised function as a string"
}}
'''

In [ ]:
print(python_engineer_template.format(code='aoeusnth'))

In [ ]:
code_string = '''
def sort(a):
    return a.sorted()
'''

user_prompt = python_engineer_template.format(code = code_string)


response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user",
         "content": user_prompt},
    ],
    # response_format={"type": "json_object"}
)

print(response.choices[0].message.content)

In [ ]:
text_output = response.choices[0].message.content

In [ ]:
data = json.loads(text_output)

In [ ]:
for i in data.get('issues'):
    print(i)

In [ ]:
print(data.get('improved_code'))

### Beware that the output may not always be *exactly* json

In [ ]:
#
# Try again with gemma3 model
# and use the below if there is a json block starting with ```json

if '```' in text_output:
    text_output = text_output.replace('```json\n','').replace('```','')
print(text_output)